In [1]:
import numpy as np
import sys
import time
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
sys.path.extend(['C:\\Users\Zack\\PycharmProjects\\Barrier3D', 'C:\\Users\\Zack\\PycharmProjects\\brie', 'C:\\Users\\Zack\\PycharmProjects\\CASCADE'])
# datadir = ".\\B3D_Inputs"
# Path(datadir)
datadir = "C:\\Users\\Zack\\PycharmProjects\\CASCADE\\B3D_Inputs\\"
from CASCADE import Cascade
from scripts import CASCADE_plotters as CASCADE_Plt

iB3D = 0
total_time = 100
cascade = Cascade(
    datadir,
    name="nourish_now_test",
    sea_level_rise_rate=0.007,
    alongshore_section_count=2,
    time_step_count=total_time,
    num_cores=8,
    roadway_management_module=False,
    alongshore_transport_module=False,
    beach_nourishment_module=True,
    community_dynamics_module=True,
    artificial_max_dune_ele=3.7,  # dune rebuilt to this height when specified
    nourishment_interval=None,  # yrs
    nourishment_volume=100,  # m^3/m -- 100 should produce about 20 m of shoreline progradation
    number_of_communities=1, 
)

nt = 50
bw_threshold = 30  # must be greater than 10 m
dune_threshold = 0.5

for _ in tqdm(range(nt-1)):
    
    cascade.update()
    if cascade.road_break or cascade.b3d_break:
        break
        
    t = cascade.barrier3d[iB3D].time_index
    if cascade.nourishments[iB3D].beach_width[t-1] < bw_threshold:
        cascade.nourish_now[iB3D] = 1
        
    DuneDomainCrest = cascade.barrier3d[iB3D].DuneDomain[t-1, :, :].max(
            axis=1
        )  # Maximum height of each row in DuneDomain
    if np.mean(DuneDomainCrest) * 10 < dune_threshold:   
        cascade.rebuild_dune_now[iB3D] = 1

In [ ]:
nourish_indices=np.argwhere(cascade.chom[0]._mgmt._nourishtime==1)
nourish_star=cascade.nourishments[iB3D]._nourishment_volume_TS[nourish_indices]


fig=plt.figure(figsize=(12,8))
plt.subplot(2,2,1)
plt.plot(cascade.nourishments[iB3D].beach_width, label="beach width")
plt.plot(cascade.chom[0]._agentsame._Ebw,label="expected beach width")
plt.legend(bbox_to_anchor=(0.35, 0.5))
plt.title("beach width")
plt.ylabel("m")
plt.xlim(1,49)

plt.subplot(2,2,2)
plt.plot(cascade.nourishments[iB3D]._nourishment_volume_TS,linewidth=3, label="nourish volume cascade")
plt.scatter(nourish_indices,nourish_star,s=40,c='orange', marker="v", label="nourish scheduled chom")
plt.title("nourishment volume")
plt.legend(bbox_to_anchor=(0.35, 0.5))
plt.ylabel("m^3/m")
plt.xlabel("years")
plt.xlim(1,49)

plt.subplot(2,2,3)
plt.plot(cascade.nourishments[iB3D]._rebuild_dune_volume_TS)
plt.title("dune sand volume")
plt.ylabel("m^3")
plt.xlabel("years")
fig.tight_layout()
plt.xlim(1,49)

plt.subplot(2,2,4)
plt.plot(cascade.chom[0]._agentsame._E_ER, label="perceived erosion rate")
plt.plot(cascade.chom[0].bw_erosion_rate, label="instantaneous erosion rate")
plt.title("erosion rate")
plt.ylabel("m/yr")
plt.legend(bbox_to_anchor=(0.35, 0.5))
plt.xlabel("years")
fig.tight_layout()
plt.xlim(1,49)


In [ ]:
nourish_indices=np.argwhere(cascade.chom[0]._mgmt._nourishtime==1)
nourish_star=cascade.nourishments[iB3D]._nourishment_volume_TS[nourish_indices]


fig=plt.figure(figsize=(12,8))
plt.subplot(2,2,1)
plt.plot(cascade.chom[0]._agent_oceanfront._beta_x="expected beach width")
plt.plot(cascade.chom[0]._agentsame._Ebw,label="expected beach width")

plt.legend(bbox_to_anchor=(0.35, 0.5))
plt.title("beach width")
plt.ylabel("m")
plt.xlim(1,49)

plt.subplot(2,2,2)
plt.plot(cascade.nourishments[iB3D]._nourishment_volume_TS,linewidth=3, label="nourish volume cascade")
plt.scatter(nourish_indices,nourish_star,s=40,c='orange', marker="v", label="nourish scheduled chom")
plt.title("nourishment volume")
plt.legend(bbox_to_anchor=(0.35, 0.5))
plt.ylabel("m^3/m")
plt.xlabel("years")
plt.xlim(1,49)

plt.subplot(2,2,3)
plt.plot(cascade.nourishments[iB3D]._rebuild_dune_volume_TS)
plt.title("dune sand volume")
plt.ylabel("m^3")
plt.xlabel("years")
fig.tight_layout()
plt.xlim(1,49)

plt.subplot(2,2,4)
plt.plot(cascade.chom[0]._agentsame._E_ER, label="perceived erosion rate")
plt.plot(cascade.chom[0].bw_erosion_rate, label="instantaneous erosion rate")
plt.title("erosion rate")
plt.ylabel("m/yr")
plt.legend(bbox_to_anchor=(0.35, 0.5))
plt.xlabel("years")
fig.tight_layout()
plt.xlim(1,49)
